In [1]:
import pandas as pd
data = pd.read_csv("fashion_products.csv")
data.head(10)

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M
5,98,6,Dress,Adidas,Men's Fashion,47,1.379566,Yellow,L
6,16,7,Jeans,Gucci,Men's Fashion,37,1.356750,White,XL
7,63,8,Sweater,Zara,Kids' Fashion,64,4.360303,Blue,XL
8,96,9,Sweater,H&M,Men's Fashion,53,4.466182,Green,XL
9,36,10,T-shirt,Zara,Kids' Fashion,55,4.093234,White,XL


In [2]:
data.isnull().sum()

,0
User ID,0
Product ID,0
Product Name,0
Brand,0
Category,0
Price,0
Rating,0
Color,0
Size,0


In [3]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2457391 sha256=205722e2a5674ceb6e441ab4edab788f923b1bbcac8ea2d439bd3a438cb981c1
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [5]:
# Downgrade numpy to a compatible version
%pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 133.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [6]:
from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [8]:
# This cell loads the data into the 'data' DataFrame.
# Please run this cell BEFORE running the content-based filtering cell (ESg6yw6HZ_sx).
import pandas as pd
data = pd.read_csv("fashion_products.csv")
print("Data loaded successfully. First 5 rows:")
display(data.head())

Data loaded successfully. First 5 rows:


,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset

# Assuming 'data' DataFrame is already loaded from your CSV
# Example:
# data = pd.read_csv("dataset.csv")

# --- 1️⃣ Prepare content DataFrame ---
content_df = data[['Product ID', 'Product Name', 'Brand',
                   'Category', 'Color', 'Size']].copy()

# Combine content into a single string
content_df['Content'] = content_df.apply(
    lambda row: ' '.join(row.dropna().astype(str)),
    axis=1
)

# --- 2️⃣ TF-IDF Vectorization ---
tfidf_vectorizer = TfidfVectorizer()
content_matrix = tfidf_vectorizer.fit_transform(content_df['Content'])

# Calculate similarity
content_similarity = linear_kernel(content_matrix, content_matrix)

# --- 3️⃣ Prepare Surprise dataset for collaborative filtering ---
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(
    data[['User ID', 'Product ID', 'Rating']],
    reader
)

# --- 4️⃣ Content-based recommendation function ---
def get_content_based_recommendations(product_id, top_n=5):
    matches = content_df[content_df['Product ID'] == product_id].index
    if len(matches) == 0:
        raise ValueError(f"Product ID {product_id} not found.")

    index = matches[0]
    similarity_scores = content_similarity[index]

    # Sort and get top N most similar products
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    recommendations = content_df.loc[similar_indices, 'Product ID'].values

    return recommendations

# --- Example usage ---
example_product_id = data['Product ID'].iloc[0]  # Pick first product from dataset
print(f"Recommendations for Product ID {example_product_id}:")
print(get_content_based_recommendations(example_product_id, top_n=5))


Recommendations for Product ID 1:
[  5   6   3 829 489]


In [10]:
get_collaborative_filtering_recommendations(79,10)

NameError: name 'get_collaborative_filtering_recommendations' is not defined

In [13]:
from surprise import SVD
from surprise.model_selection import train_test_split
import pandas as pd

# --- 1️⃣ Train-test split ---
trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=42)

# --- 2️⃣ Train a collaborative filtering model (SVD) ---
model = SVD()
model.fit(trainset)

# --- 3️⃣ Function for collaborative filtering recommendations ---
def get_collaborative_filtering_recommendations(user_id, top_n=10):
    # Get all product IDs
    all_product_ids = data['Product ID'].unique()

    # Get products the user has already rated
    rated_products = data[data['User ID'] == user_id]['Product ID'].unique()

    # Predict ratings for products the user hasn't rated yet
    predictions = []
    for pid in all_product_ids:
        if pid not in rated_products:
            pred = model.predict(user_id, pid)
            predictions.append((pid, pred.est))

    # Sort predictions by estimated rating (highest first)
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Return top_n product IDs
    recommended_product_ids = [pid for pid, _ in predictions[:top_n]]
    return recommended_product_ids

# --- Example usage ---
print(get_collaborative_filtering_recommendations(79, 10))


[104, 656, 731, 194, 449, 569, 205, 230, 609, 800]


In [14]:
# Hybrid Approach
def get_hybrid_recommendations(user_id, product_id, top_n):
    content_based_recommendations = get_content_based_recommendations(product_id, top_n)
    collaborative_filtering_recommendations = get_collaborative_filtering_recommendations(user_id, top_n)
    hybrid_recommendations = list(set(content_based_recommendations + collaborative_filtering_recommendations))
    return hybrid_recommendations[:top_n]

In [15]:
user_id = 79
product_id = 5
top_n = 10
recommendations = get_hybrid_recommendations(user_id, product_id, top_n)

print(f"Hybrid Recommendations for User {user_id} based on Product {product_id}:")
for i, recommendation in enumerate(recommendations):
    print(f"{i + 1}. Product ID: {recommendation}")

Hybrid Recommendations for User 79 based on Product 5:
1. Product ID: 1095
2. Product ID: 1032
3. Product ID: 105
4. Product ID: 1386
5. Product ID: 236
6. Product ID: 525
7. Product ID: 877
8. Product ID: 1520
9. Product ID: 1720
10. Product ID: 1019
